In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.statespace.varmax import VARMAX
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
dataset = pd.read_csv("Ts-OHLC.csv")
dataset

,Open,High,Low,Close
0,1410.60,1427.90,1408.30,1415.10
1,1421.00,1626.60,1416.15,1607.40
2,1632.55,1673.90,1613.05,1626.20
3,1627.75,1627.75,1574.60,1579.05
4,1580.00,1639.50,1565.50,1595.65
...,...,...,...,...
2220,202.90,207.80,201.35,205.00
2221,206.00,206.85,202.05,202.95
2222,203.90,203.90,199.35,201.00
2223,200.00,222.00,196.00,218.35


In [2]:
column = "Close"

In [3]:
from sklearn.preprocessing import MinMaxScaler
Ms=MinMaxScaler()
data1 = Ms.fit_transform(dataset[[column]])
print("Len:" , data1.shape)

Len: (2225, 1)


In [4]:
training_size = round(len(data1) * 0.95)
print(training_size)

X_train = data1[:training_size]
X_test = data1[training_size:]
print("X_train length:", X_train.shape)
print("X_test length:", X_test.shape)

y_train = data1[:training_size]
y_test = data1[training_size:]
print("y_train length:", y_train.shape)
print("y_test length:", y_test.shape)

2114
X_train length: (2114, 1)
X_test length: (111, 1)
y_train length: (2114, 1)
y_test length: (111, 1)


In [8]:
pip install pmdarima


   -------------------------------------- 613.6/613.6 kB 177.9 kB/s eta 0:00:00


In [10]:
from pmdarima.arima import auto_arima

def combination(dataset, listt):
    datasetTwo = dataset[listt].values  # Directly slice the numpy array
    test_obs = 28
    train = datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]

    performance = {"Model": [], "RMSE": [], "MaPe": [], "Lag": [], "Test": []}

    # Select the best order based on AIC/BIC
    model = auto_arima(train[:, 0], seasonal=False, trace=True, error_action='ignore', suppress_warnings=True)
    best_order = model.order

    # Fit VARMAX model with the best selected order
    result = VARMAX(train, order=best_order).fit()

    # Forecast
    pred = result.forecast(steps=28)

    # Convert predictions to DataFrame
    preds = pd.DataFrame(pred, columns=listt)
    preds.to_csv("varma_forecasted_{}.csv".format(test_obs))

    # Calculate performance metrics
    rmse = round(mean_squared_error(test[:, 0], pred[:, 0], squared=False))
    mape = mean_absolute_percentage_error(test[:, 0], pred[:, 0])

    # Store performance metrics
    performance["Model"].append(listt[0])  # Only store the performance for the first variable
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append(best_order)
    performance["Test"].append(test_obs)

    # Create DataFrame for performance metrics
    perf = pd.DataFrame(performance)
    return perf, result, pred

listt = ["Close", "High", "Open", "Low"]
perf, result, pred = combination(dataset, listt)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=19918.424, Time=2.26 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=19916.253, Time=0.07 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=19913.576, Time=0.55 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=19913.414, Time=0.47 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=19915.515, Time=0.06 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=19915.113, Time=1.31 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=19914.938, Time=0.59 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=19916.661, Time=0.90 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=19912.546, Time=0.17 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=19914.265, Time=0.97 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=19914.107, Time=0.29 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=19912.706, Time=0.18 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=19915.721, Time=1.08 sec

Best model:  ARIMA(0,1,1)(0,0,0)[0]          
Total fit time: 9.007 seconds


C:\Users\Admin\anaconda3\envs\AIcoures\lib\site-packages\statsmodels\base\model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


In [11]:
perf

,Model,RMSE,MaPe,Lag,Test
0,Close,131,0.621583,"(0, 1, 1)",28
